#  Show the tradeoffs between Spacy vs ChatGPT for entity recognition


Requirements
```
python -m spacy download en_core_web_sm
```
```python
import nltk
nltk.download('vader_lexicon')
```

In [1]:
from pydantic import BaseModel, Field

from promptedgraphs.config import Config
from promptedgraphs.ideation import brainstorm
from promptedgraphs.vis import render_entities
from promptedgraphs.config import Config, load_config
from promptedgraphs.entity_recognition import extract_entities
from promptedgraphs.models import EntityReference
import spacy
from nltk.sentiment import SentimentIntensityAnalyzer
from typing import NamedTuple, Dict, List
import asyncio
import datetime

_ = load_config()

In [2]:
async def label_sentiment(text_of_reviews, model=None, temperature=0):

    labels = {
        "POSITIVE": "A postive review of a product or service.",
        "NEGATIVE": "A negative review of a product or service.",
        "NEUTRAL": "A neutral review of a product or service.",
    }

    ents = []
    async for msg in extract_entities(
        name="sentiment",
        description="Sentiment Analysis of Customer Reviews",
        text=text_of_reviews,
        temperature=temperature,
        labels=labels,
        model=model,
        config=Config(),
        include_reason=False,
    ):
        ents.append(msg)
    return ents


text_of_reviews = """
1. "I absolutely love this product. It's been a game changer!"
2. "The service was quite poor and the staff was rude."
3. "The item is okay. Nothing special, but it gets the job done."
""".strip()

## Two GPT approaches

In [3]:
ents = await label_sentiment(text_of_reviews, model='gpt-3.5-turbo-1106', temperature=0)

render_entities(text_of_reviews, ents)
display(ents[-1])

Usage(model=gpt-3.5-turbo-1106, prompt_tokens=380, completion_tokens=122, duration=4.1737, cost=0.000624)

In [4]:
ents2 = await label_sentiment(text_of_reviews, model='gpt-4-1106-preview')

render_entities(text_of_reviews, ents2)
display(ents2[-1])

Usage(model=gpt-4-1106-preview, prompt_tokens=373, completion_tokens=125, duration=10.2242, cost=0.007480)

# Playground for Spacy implementation

In [5]:
nlp = spacy.load("en_core_web_sm")

async def extract_entities_spacy(name: str, description: str, text: str, labels: Dict[str, str]) -> List[EntityReference]:
    # Load spaCy model for sentence tokenization
    
    # Initialize the sentiment analyzer
    sia = SentimentIntensityAnalyzer()

    # Tokenize the text into sentences
    doc = nlp(text)
    entities = []

    for sentence in doc.sents:
        # Analyze sentiment of the sentence
        sentiment_score = sia.polarity_scores(sentence.text)
        
        # Determine the sentiment label
        if sentiment_score["neu"] >= 0.99:
            continue
        if sentiment_score["compound"] >= 0.05:
            sentiment_label = "POSITIVE"
        elif sentiment_score["compound"] <= -0.05:
            sentiment_label = "NEGATIVE"
        else:
            sentiment_label = "NEUTRAL"

        # Create EntityReference
        entity = EntityReference(
            start=sentence.start_char, 
            end=sentence.end_char, 
            label=sentiment_label, 
            text=sentence.text, 
            reason=str(sentiment_score)
        )
        entities.append(entity)
    
    return entities

In [9]:
# Compute time calculated assuming AWS ondemand prices
# t4g.medium	$0.0336	2	4 GiB	EBS Only	Up to 5 Gigabit
compute_cost_second = 0.0336 / 60 / 60 

9.333333333333333e-06

In [8]:
from promptedgraphs.llms.openai_token_counter import Usage

st = datetime.datetime.now(tz=datetime.timezone.utc)


spacy_usage = Usage(model='spacy')
spacy_usage.start()

ents_spacy = await extract_entities_spacy(
    name="sentiment",
    description="Sentiment Analysis of Customer Reviews",
    text=text_of_reviews,
    labels={
        "POSITIVE": "A positive review of a product or service.",
        "NEGATIVE": "A negative review of a product or service.",
        "NEUTRAL": "A neutral review of a product or service.",
    }
)
spacy_usage.end()
print(spacy_usage)


Usage(model=spacy, prompt_tokens=0, completion_tokens=0, duration=0.0172, cost=0.000000)


In [22]:
import pandas as pd

costs = pd.DataFrame([
    spacy_usage.dict(), ents[-1].dict(), ents2[-1].dict()
]).set_index('model').rename(columns={'cost':'api-cost'})
costs.loc[:, 'cost'] = (costs['api-cost'] + costs['duration'] * compute_cost_second).round(6)

costs

,prompt_tokens,completion_tokens,duration,api-cost,cost
model,,,,,
spacy,0,0,0.017180,0.000000,0.000000
gpt-3.5-turbo-1106,380,122,4.173707,0.000624,0.000663
gpt-4-1106-preview,373,125,10.224201,0.007480,0.007575


In [10]:
spacy_usage.duration*compute_cost_second

1.6034634908040364e-07

In [11]:
ents[-1].duration*compute_cost_second

3.895460096995036e-05

In [24]:
from promptedgraphs.vis import get_colors, get_fields

colors = get_colors(get_fields(ents[:-1]))

In [25]:
render_entities(text_of_reviews, ents, color_dict=colors)

In [27]:
render_entities(text_of_reviews, ents2, color_dict=colors)

In [26]:
render_entities(text_of_reviews, ents_spacy, color_dict=colors)

In [33]:
!poetry add torch

Using version ^2.2.0 for torch

Updating dependencies
Resolving dependencies... (3.0s)

Package operations: 14 installs, 0 updates, 0 removals

  • Installing nvidia-cublas-cu12 (12.1.3.1): Pending...
  • Installing nvidia-cublas-cu12 (12.1.3.1): Downloading... 0%
  • Installing nvidia-cublas-cu12 (12.1.3.1): Downloading... 10%
  • Installing nvidia-cublas-cu12 (12.1.3.1): Downloading... 20%
  • Installing nvidia-cublas-cu12 (12.1.3.1): Downloading... 30%
  • Installing nvidia-cublas-cu12 (12.1.3.1): Downloading... 40%
  • Installing nvidia-cublas-cu12 (12.1.3.1): Downloading... 50%
  • Installing nvidia-cublas-cu12 (12.1.3.1): Downloading... 60%
  • Installing nvidia-cublas-cu12 (12.1.3.1): Downloading... 70%
  • Installing nvidia-cublas-cu12 (12.1.3.1): Downloading... 80%
  • Installing nvidia-cublas-cu12 (12.1.3.1): Downloading... 90%
  • Installing nvidia-cublas-cu12 (12.1.3.1): Downloading... 100%
  • Installing nvidia-cublas-cu12 (12.1.3.1): Installing...
Installing /home/skruz/.

  • Installing nvidia-cuda-cupti-cu12 (12.1.105): Pending...
  • Installing nvidia-cuda-nvrtc-cu12 (12.1.105): Pending...
  • Installing nvidia-cuda-runtime-cu12 (12.1.105): Pending...
  • Installing nvidia-cudnn-cu12 (8.9.2.26): Downloading... 0%
  • Installing nvidia-cufft-cu12 (11.0.2.54): Pending...
  • Installing nvidia-curand-cu12 (10.3.2.106): Pending...
  • Installing nvidia-cusolver-cu12 (11.4.5.107): Pending...
  • Installing nvidia-nccl-cu12 (2.19.3): Pending...
  • Installing nvidia-nvtx-cu12 (12.1.105): Pending...
  • Installing sympy (1.12): Pending...
  • Installing triton (2.2.0): Pending...
  • Installing networkx (3.2.1)
  • Installing nvidia-cuda-cupti-cu12 (12.1.105): Pending...
  • Installing nvidia-cuda-nvrtc-cu12 (12.1.105): Pending...
  • Installing nvidia-cuda-runtime-cu12 (12.1.105): Pending...
  • Installing nvidia-cudnn-cu12 (8.9.2.26): Downloading... 0%
  • Installing nvidia-cufft-cu12 (11.0.2.54): Pending...
  • Installing nvidia-curand-cu12 (10.3.2.106): 

  • Installing nvidia-cuda-runtime-cu12 (12.1.105): Downloading... 0%
  • Installing nvidia-cudnn-cu12 (8.9.2.26): Downloading... 10%
  • Installing nvidia-cufft-cu12 (11.0.2.54): Pending...
  • Installing nvidia-curand-cu12 (10.3.2.106): Pending...
  • Installing nvidia-cusolver-cu12 (11.4.5.107): Downloading... 0%
  • Installing nvidia-nccl-cu12 (2.19.3): Pending...
  • Installing nvidia-nvtx-cu12 (12.1.105): Pending...
  • Installing sympy (1.12): Pending...
  • Installing triton (2.2.0): Pending...
  • Installing nvidia-cuda-nvrtc-cu12 (12.1.105): Downloading... 0%
  • Installing nvidia-cuda-runtime-cu12 (12.1.105): Downloading... 0%
  • Installing nvidia-cudnn-cu12 (8.9.2.26): Downloading... 10%
  • Installing nvidia-cufft-cu12 (11.0.2.54): Pending...
  • Installing nvidia-curand-cu12 (10.3.2.106): Pending...
  • Installing nvidia-cusolver-cu12 (11.4.5.107): Downloading... 0%
  • Installing nvidia-nccl-cu12 (2.19.3): Pending...
  • Installing nvidia-nvtx-cu12 (12.1.105): Pending..

  • Installing sympy (1.12): Downloading... 0%
  • Installing triton (2.2.0): Downloading... 0%
  • Installing nvidia-nvtx-cu12 (12.1.105): Downloading... 20%
  • Installing sympy (1.12): Downloading... 0%
  • Installing triton (2.2.0): Downloading... 0%
  • Installing nvidia-cudnn-cu12 (8.9.2.26): Downloading... 10%
  • Installing nvidia-cufft-cu12 (11.0.2.54): Downloading... 0%
  • Installing nvidia-curand-cu12 (10.3.2.106): Downloading... 0%
  • Installing nvidia-cusolver-cu12 (11.4.5.107): Downloading... 0%
  • Installing nvidia-nccl-cu12 (2.19.3): Downloading... 0%
  • Installing nvidia-nvtx-cu12 (12.1.105): Downloading... 20%
  • Installing sympy (1.12): Downloading... 0%
  • Installing triton (2.2.0): Downloading... 0%
  • Installing nvidia-cuda-runtime-cu12 (12.1.105): Downloading... 10%
  • Installing nvidia-cudnn-cu12 (8.9.2.26): Downloading... 10%
  • Installing nvidia-cufft-cu12 (11.0.2.54): Downloading... 0%
  • Installing nvidia-curand-cu12 (10.3.2.106): Downloading... 0%

Installing /home/skruz/.cache/pypoetry/virtualenvs/promptedgraphs-5vPvCOs7-py3.10/lib/python3.10/site-packages/nvidia/__init__.py over existing file
  • Installing sympy (1.12): Downloading... 0%
  • Installing triton (2.2.0): Downloading... 0%
  • Installing nvidia-nvtx-cu12 (12.1.105): Downloading... 41%
  • Installing sympy (1.12): Downloading... 0%
  • Installing triton (2.2.0): Downloading... 0%
  • Installing sympy (1.12): Downloading... 0%
  • Installing triton (2.2.0): Downloading... 0%
  • Installing nvidia-nvtx-cu12 (12.1.105): Downloading... 100%
  • Installing sympy (1.12): Downloading... 0%
  • Installing triton (2.2.0): Downloading... 0%
  • Installing sympy (1.12): Downloading... 0%
  • Installing triton (2.2.0): Downloading... 0%
  • Installing nvidia-nvtx-cu12 (12.1.105): Installing...
  • Installing sympy (1.12): Downloading... 0%
  • Installing triton (2.2.0): Downloading... 0%
Installing /home/skruz/.cache/pypoetry/virtualenvs/promptedgraphs-5vPvCOs7-py3.10/lib/pyth

...

       621│             def func_wrapper(*args, **kwargs):
    →  622│                 return func(*args, **kwargs)
       623│             # Avoid closing the file as long as the wrapper is alive,
       624│             # see issue #18879.
       625│             func_wrapper._closer = self._closer
       626│             a = func_wrapper

The following error occurred when trying to handle this error:


  ProtocolError

  ("Connection broken: OSError(28, 'No space left on device')", OSError(28, 'No space left on device'))

  at ~/.local/pipx/venvs/poetry/lib/python3.10/site-packages/urllib3/response.py:729 in _error_catcher
       725│                 raise ReadTimeoutError(self._pool, None, "Read timed out.") from e  # type: ignore[arg-type]
       726│ 
       727│             except (HTTPException, OSError) as e:
       728│                 # This includes IncompleteRead.
    →  729│                 raise ProtocolError(f"Connection broken: {e!r}", e) from e
       730│ 
       731

       725│                 raise ReadTimeoutError(self._pool, None, "Read timed out.") from e  # type: ignore[arg-type]
  • Installing nvidia-cufft-cu12 (11.0.2.54): Failed
  • Installing nvidia-curand-cu12 (10.3.2.106): Failed

  OSError

  [Errno 28] No space left on device

  at ~/.local/pipx/venvs/poetry/lib/python3.10/site-packages/cachecontrol/caches/file_cache.py:132 in _write
      128│ 
      129│         with self.lock_class(path + ".lock"):
      130│             # Write our actual file
      131│             with _secure_open_write(path, self.filemode) as fh:
    → 132│                 fh.write(data)
      133│ 
      134│     def _delete(self, key: str, suffix: str) -> None:
      135│         name = self._fn(key) + suffix
      136│         if not self.forever:

The following error occurred when trying to handle this error:


  ProtocolError

  ("Connection broken: OSError(28, 'No space left on device')", OSError(28, 'No space left on device'))

  at ~/.local/pipx/venvs/

       727│             except (HTTPException, OSError) as e:
       728│                 # This includes IncompleteRead.
    →  729│                 raise ProtocolError(f"Connection broken: {e!r}", e) from e
       730│ 
       731│             # If no exception is thrown, we should avoid cleaning up
       732│             # unnecessarily.
       733│             clean_exit = True

The following error occurred when trying to handle this error:


  ChunkedEncodingError

  ("Connection broken: OSError(28, 'No space left on device')", OSError(28, 'No space left on device'))

  at ~/.local/pipx/venvs/poetry/lib/python3.10/site-packages/requests/models.py:818 in generate
       814│             if hasattr(self.raw, "stream"):
       815│                 try:
       816│                     yield from self.raw.stream(chunk_size, decode_content=True)
       817│                 except ProtocolError as e:
    →  818│                     raise ChunkedEncodingError(e)
       819│              

       814│             if hasattr(self.raw, "stream"):
  • Installing nvidia-cufft-cu12 (11.0.2.54): Failed
  • Installing nvidia-curand-cu12 (10.3.2.106): Failed

  OSError

  [Errno 28] No space left on device

  at ~/.local/pipx/venvs/poetry/lib/python3.10/site-packages/cachecontrol/caches/file_cache.py:132 in _write
      128│ 
      129│         with self.lock_class(path + ".lock"):
      130│             # Write our actual file
      131│             with _secure_open_write(path, self.filemode) as fh:
    → 132│                 fh.write(data)
      133│ 
      134│     def _delete(self, key: str, suffix: str) -> None:
      135│         name = self._fn(key) + suffix
      136│         if not self.forever:

The following error occurred when trying to handle this error:


  ProtocolError

  ("Connection broken: OSError(28, 'No space left on device')", OSError(28, 'No space left on device'))

  at ~/.local/pipx/venvs/poetry/lib/python3.10/site-packages/urllib3/response.py:729 in _e

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



# Evaluate performance based on


BERT language model

- Accuracy
    - Precision
    - Recall

In [32]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch
from collections import defaultdict

# Load the BERT model and tokenizer
model_name = "dbmdz/bert-large-cased-finetuned-conll03-english"
model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

async def extract_entities_bert(name: str, description: str, text: str, labels: Dict[str, str]) -> List[EntityReference]:
    # Tokenize the text and convert to tensor
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # Predict entities using BERT
    with torch.no_grad():
        outputs = model(**inputs).logits
    predictions = torch.argmax(outputs, dim=2)

    # Map predictions to entity labels
    tokenized_text = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    entities = defaultdict(list)

    for token, prediction in zip(tokenized_text, predictions[0].numpy()):
        label = model.config.id2label[prediction]
        if label != "O":  # O means no entity
            entities[label].append(token)

    # Post-process to combine subword tokens and convert to EntityReference
    processed_entities = []
    for label, tokens in entities.items():
        entity_text = tokenizer.convert_tokens_to_string(tokens)
        entity = EntityReference(
            label=label,
            text=entity_text
        )
        processed_entities.append(entity)

    return processed_entities


ModuleNotFoundError: No module named 'torch'

In [ ]:
ents_api